In [1]:
from datasets.get_datasets import *
import time
from src.genetic import *

from src.predicate import *
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

# Defining BK and dataset

In [2]:
bk = {
      'nell_sports': ['athleteledsportsteam(+athlete,+sportsteam).',
              'athleteledsportsteam(+athlete,-sportsteam).',
              'athleteledsportsteam(-athlete,+sportsteam).',
              'athleteplaysforteam(+athlete,+sportsteam).',
              'athleteplaysforteam(+athlete,-sportsteam).',
              'athleteplaysforteam(-athlete,+sportsteam).',
              'athleteplaysinleague(+athlete,+sportsleague).',
              'athleteplaysinleague(+athlete,-sportsleague).',
              'athleteplaysinleague(-athlete,+sportsleague).',
              'athleteplayssport(+athlete,+sport).',
              'athleteplayssport(+athlete,-sport).',
              'athleteplayssport(-athlete,+sport).',
              'teamalsoknownas(+sportsteam,+sportsteam).',
              'teamalsoknownas(+sportsteam,-sportsteam).',
              'teamalsoknownas(-sportsteam,+sportsteam).',
              'teamplaysagainstteam(+sportsteam,+sportsteam).',
              'teamplaysagainstteam(+sportsteam,-sportsteam).',
              'teamplaysagainstteam(-sportsteam,+sportsteam).',
              'teamplaysinleague(+sportsteam,+sportsleague).',
              'teamplaysinleague(+sportsteam,-sportsleague).',
              'teamplaysinleague(-sportsteam,+sportsleague).',
              'teamplayssport(+sportsteam,+sport).',
              'teamplayssport(+sportsteam,-sport).',
              'teamplayssport(-sportsteam,+sport).'],
      'nell_finances': ['countryhascompanyoffice(+country,+company).',
                        'countryhascompanyoffice(+country,-company).',
                        'countryhascompanyoffice(-country,+company).',
                        'companyeconomicsector(+company,+sector).',
                        'companyeconomicsector(+company,-sector).',
                        'companyeconomicsector(-company,+sector).',
                        'economicsectorcompany(+sector,-company).',
                        'economicsectorcompany(-sector,+company).',
                        'companyceo(+company,+person).',
                        'companyceo(+company,-person).',
                        'companyceo(-company,+person).',
                        'companyalsoknownas(+company,+company).',
                        'companyalsoknownas(+company,-company).',
                        'companyalsoknownas(-company,+company).',
                        'cityhascompanyoffice(+city,+company).',
                        'cityhascompanyoffice(+city,-company).',
                        'cityhascompanyoffice(-city,+company).',
                        'acquired(+company,+company).',
                        'acquired(+company,-company).',
                        'acquired(-company,+company).',
                        'bankbankincountry(+person,+country).',
                        'bankbankincountry(+person,-country).',
                        'bankbankincountry(-person,+country).',
                        'bankboughtbank(+company,+company).',
                        'bankboughtbank(+company,-company).',
                        'bankboughtbank(-company,+company).',
                        'bankchiefexecutiveceo(+company,+person).',
                        'bankchiefexecutiveceo(+company,-person).',
                        'bankchiefexecutiveceo(-company,+person).']}


In [3]:
bk_source = bk['nell_sports']
bk_target = bk['nell_finances']

In [4]:
nsports_dataset = datasets.load('nell_sports', bk_source, target='teamplayssport', seed=441773) #facts, pos, neg

In [5]:
train_facts_source = []
train_pos_source = []
train_neg_source = []
for i in range(0, len(nsports_dataset[0])):
    train_facts_source.extend(nsports_dataset[0][i])
    train_pos_source.extend(nsports_dataset[1][i])
    train_neg_source.extend(nsports_dataset[2][i])

In [6]:
target = ['teamplayssport']
background_knowledge_src = boostsrl.modes(bk_source, target, useStdLogicVariables=False, maxTreeDepth=3, nodeSize=2, numOfClauses=8)

In [7]:
model_src = boostsrl.train(background_knowledge_src, train_pos_source, train_neg_source, train_facts_source, trees=10)

In [8]:
structured_src = []
for i in range(0, 10):
    try:
        structured_src.append(model_src.get_structured_tree(treenumber=i+1).copy())
    except:
        structured_src.append(model_src.get_structured_tree(treenumber='combine').copy())

In [9]:
structured_src

[['teamplayssport(A, B)',
  {'': 'athleteplayssport(C, B), athleteledsportsteam(C, D)',
   'true': 'teamplaysinleague(A, E), athleteplaysinleague(C, E)',
   'true,false': 'teamplaysinleague(A, F), athleteplaysinleague(G, F)',
   'true,false,true': 'athleteledsportsteam(G, D)',
   'true,false,false': 'teamalsoknownas(A, H), teamplaysagainstteam(A, I)',
   'false': 'athleteplayssport(J, B), athleteplaysinleague(J, K)',
   'false,true': 'teamplaysinleague(A, K)',
   'false,true,false': 'teamplaysagainstteam(L, A)'},
  {'true,true': [0.261, 14, 177],
   'true,false,true,true': [2.711, 40, 9],
   'true,false,true,false': [3.328, 16, 36],
   'true,false,false,true': [1.581, 5, 5],
   'true,false,false,false': [6.368, 56, 147],
   'false,true,true': [0.0, 17, 0],
   'false,true,false,true': [2.519, 11, 15],
   'false,true,false,false': [1.323, 14, 2],
   'false,false': [0.159, 224, 6]}],
 ['teamplayssport(A, B)',
  {'': 'athleteplayssport(C, B), athleteledsportsteam(C, D)',
   'true': 'teampl

In [10]:
#pred_target é : pred_target = [('movie', '+,-'), ('director', '+'),...]
import re
pred_target = []
for pred in bk_target:
    modes = ','.join([pred[occur.start()] for occur in re.finditer('[+\-]', pred)])
    pred_target.append((pred.split('(')[0], modes))
    


In [11]:
def get_branch(curr_value, next_value):
    if curr_value == '': 
        return next_value
    return '{},{}'.format(curr_value, next_value)

def define_individual(structured_tree, tree_number):
    individual_tree = []
    target = structured_tree[0]
    nodes = structured_tree[1]
    for values, node in nodes.items():
        if values == '': 
            branch = '{} :- {}.'.format(target, node)
        else: branch = '{}.'.format(node)
        left_branch = 'true' if get_branch(values, 'true') in nodes  else 'false'
        right_branch = 'true' if get_branch(values, 'false') in nodes else 'false'
        individual_tree.append('{};{};{};{};{}'.format(tree_number, values, 
                                                  branch, left_branch, right_branch))
    return individual_tree

src_struct = copy.deepcopy(structured_src)
new_src_struct = []
for i in range(0, len(src_struct)):
    new_src_struct.append(define_individual(src_struct[i], i))


# Target dataset

In [12]:
nfinances_dataset = datasets.load('nell_finances', bk_target, target='companyeconomicsector', seed=441773) #facts, pos, neg

In [13]:
import numpy as np
from sklearn.model_selection import KFold
kf = KFold(n_splits=4)

train_facts = []
train_pos = []
train_neg = []

for i in kf.split(nfinances_dataset[0][0]):
    index = i[1]
    train_facts.append(list(np.array(nfinances_dataset[0][0])[index]))
    
for i in kf.split(nfinances_dataset[1][0]):
    index = i[1]
    train_pos.append(list(np.array(nfinances_dataset[1][0])[index]))
    train_neg.append(list(np.array(nfinances_dataset[2][0])[index]))
    

In [14]:
train_facts_target = train_facts[1:]
train_pos_target = train_pos[1:]
train_neg_target = train_neg[1:]

test_facts = train_facts[0]
test_pos = train_pos[0]
test_neg = train_neg[0]


In [15]:
start = time.time()
pred = Predicate(bk_source, bk_target, pred_target)
aa = genetic(new_src_struct, 'companyeconomicsector', 'teamplayssport', pred, train_pos_target, 
                        train_neg_target, train_facts_target,
                        NUM_GEN=5, pop_size=30)
print("FIM: ", time.time()-start)

/usr/local/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.8/site-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


MEDIA
AUC PR:  0.5
AUC ROC:  0.5
CLL:  -0.695026
PREC:  nan
RECALL:  0.0
F1:  nan
-------------------
STD
AUC PR:  0.0
AUC ROC:  0.0
CLL:  0.0
PREC:  nan
RECALL:  0.0
F1:  nan
-------------------
MEDIA
AUC PR:  0.5
AUC ROC:  0.5
CLL:  -0.695026
PREC:  nan
RECALL:  0.0
F1:  nan
-------------------
STD
AUC PR:  0.0
AUC ROC:  0.0
CLL:  0.0
PREC:  nan
RECALL:  0.0
F1:  nan
-------------------
MEDIA
AUC PR:  0.5
AUC ROC:  0.5
CLL:  -0.695087
PREC:  nan
RECALL:  0.0
F1:  nan
-------------------
STD
AUC PR:  0.0
AUC ROC:  0.0
CLL:  7.12787953508287e-05
PREC:  nan
RECALL:  0.0
F1:  nan
-------------------
MEDIA
AUC PR:  0.5
AUC ROC:  0.5
CLL:  -0.6950653333333333
PREC:  nan
RECALL:  0.0
F1:  nan
-------------------
STD
AUC PR:  0.0
AUC ROC:  0.0
CLL:  4.5433712397523836e-05
PREC:  nan
RECALL:  0.0
F1:  nan
-------------------
MEDIA
AUC PR:  0.5
AUC ROC:  0.5
CLL:  -0.695087
PREC:  nan
RECALL:  0.0
F1:  nan
-------------------
STD
AUC PR:  0.0
AUC ROC:  0.0
CLL:  7.12787953508287e-05
PREC:  nan

Exception: ('Encountered problems while running process: ', '(cd boostsrl; java -jar v1-0.jar -i -model train/models/ -test test/ -target companyeconomicsector -trees 10 -aucJarPath . > test_output.txt 2>&1)')

In [45]:
refine = []
for tree in range(0, len(aa[0].population[-1].modified_src_tree)):
    for i in aa[0].population[-1].modified_src_tree[tree]:
        tt = i.split(';')
        tt[0] = tt[0].replace(tt[0], str(tree))
        refine.append(";".join(tt))


In [84]:
aa[0].population[-1].predicate_inst.mapping_var

{'sportsteam': 'company',
 'sport': 'sector',
 'athlete': 'company',
 'sportsleague': 'sector',
 'company': 'company',
 'sector': 'sector'}

In [37]:
background_train = boostsrl.modes(bk_target, ['companyeconomicsector'], useStdLogicVariables=False, 
                                          maxTreeDepth=3, nodeSize=2, numOfClauses=8)

In [53]:
# TESTE

train_facts = []
train_pos = []
train_neg = []

for i in range(0, len(train_facts_target)):
    train_facts.extend(train_facts_target[i])
    train_pos.extend(train_pos_target[i])
    train_neg.extend(train_neg_target[i])


model_tr = boostsrl.train(background_train, train_pos_target[0], train_neg_target[0], 
                                      train_facts_target[0], refine=refine, transfer=aa[0].population[-1].transfer.transfer,
                                      trees=10)

In [54]:
test_model = boostsrl.test(model_tr, test_pos, test_neg, 
                                       test_facts, trees=10)

In [55]:
test_model.summarize_results()

{'AUC ROC': 0.5,
 'AUC PR': 0.5,
 'CLL': -0.695026,
 'Precision': [nan, 0.5],
 'Recall': 0.0,
 'F1': nan}

In [58]:
results_auc_pr = [0.5, 0.7703486666666667, 0.7703486666666667,0.770001,0.770562,0.7704886666666666,0.7704886666666666,0.7704886666666666,0.770088,0.770088,0.770088,0.770088,0.770088,0.770088,0.770088,0.770088]
results_auc_roc = [0.5, 0.7670573333333334, 0.7670573333333334,0.7669193333333334,0.767661,0.7674949999999999,0.7674949999999999,0.7674949999999999,0.7671296666666666,0.7671296666666666,0.7671296666666666,0.7671296666666666,0.7671296666666666,0.7671296666666666,0.7671296666666666,0.7671296666666666]
results_cll = [-0.695026, -0.6321996666666666, -0.6321996666666666,-0.6113069999999999,-0.6112639999999999,-0.6100530000000001,-0.6100530000000001,-0.6100530000000001,-0.6095830000000001,-0.6095830000000001,-0.6095830000000001,-0.6095830000000001,-0.6095830000000001,-0.6095830000000001,-0.6095830000000001,-0.6095830000000001]
results_prec = [None, None, None,None,None,None,None,None,None,None,None,None,None,None,None,None]
results_rec = [0.0, 0.620407,0.620407,0.620407,0.620407,0.620407,0.620407,0.620407,0.620407,0.620407,0.620407,0.620407,0.620407,0.620407,0.620407,0.620407]
results_f1 = [None, None, None,None,None,None,None,None,None,None,None,None,None,None,None,None]

std_auc_pr = [0.0, 0.19702201363525065,0.19702201363525065,0.1969788038563202,0.197247419077327,0.1971671458793838,0.1971671458793838,0.1971671458793838,0.1970646744210303,0.1970646744210303,0.1970646744210303,0.1970646744210303,0.1970646744210303,0.1970646744210303,0.1970646744210303,0.1970646744210303]
std_auc_roc = [0.0, 0.19454420426787897,0.19454420426787897,0.19477853723196048,0.19519025534249057,0.19500036353983202,0.19500036353983202,0.19500036353983202,0.19498641468289243,0.19498641468289243,0.19498641468289243,0.19498641468289243,0.19498641468289243,0.19498641468289243,0.19498641468289243,0.19498641468289243]
std_cll = [0.0, 0.08144984526825215,0.08144984526825215,0.14810376781387657,0.15028295975924882,0.20829419553282483,0.20829419553282483,0.20829419553282483,0.2087955630579028,0.2087955630579028,0.2087955630579028,0.2087955630579028,0.2087955630579028,0.2087955630579028,0.2087955630579028,0.2087955630579028]
std_prec = [None, None, None,None,None,None,None,None,None,None,None,None,None,None,None,None]
std_rec = [0.0,0.43873711784241215, 0.43873711784241215,0.43873711784241215,0.43873711784241215,0.43873711784241215,0.43873711784241215,0.43873711784241215,0.43873711784241215,0.43873711784241215,0.43873711784241215,0.43873711784241215,0.43873711784241215,0.43873711784241215,0.43873711784241215,0.43873711784241215]
std_f1 = [None, None, None,None,None,None,None,None,None,None,None,None,None,None,None,None]


In [73]:
fig = go.Figure()

fig.add_trace(go.Scatter(
        x=list(range(0, len(results_auc_pr))),
        y=results_auc_pr,name='AUC PR',
        error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=std_auc_pr,
            visible=True)
    ))

fig.add_trace(go.Scatter(
        x=list(range(0, len(results_auc_roc))),
        y=results_auc_roc,name='AUC ROC',
        error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=std_auc_roc,
            visible=True)
    ))


fig.add_annotation(x=0, y=results_auc_pr[0],showarrow=False,
            text="0.5",yshift=10, xshift=-30,
            font=dict(color='rgba(152, 0, 0, .8)'))

fig.add_annotation(x=len(results_auc_roc), y=results_auc_roc[-1],showarrow=False,
            text="0.770088",yshift=-10, xshift=5,
            font=dict(color='rgba(152, 0, 0, .8)'))

fig.add_annotation(x=len(results_auc_pr), y=results_auc_pr[-1],showarrow=False,
            text="0.7671296",yshift=0, xshift=5)
#             font=dict(color='rgba(152, 0, 0, .8)'))

fig.add_annotation(x=1, y=results_auc_roc[1],showarrow=False,
            text="0.7703486",yshift=-10, xshift=-40,
            font=dict(color='rgba(152, 0, 0, .8)'))

fig.add_annotation(x=1, y=results_auc_pr[1],showarrow=False,
            text="0.7670573",yshift=0, xshift=-40)
#             font=dict(color='rgba(152, 0, 0, .8)'))


fig.update_layout(title='AUC PR e AUC ROC ao longo das gerações')
fig.update_layout(
    xaxis_title="geracoes",
)

fig.show()


In [82]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
        x=list(range(0, len(results_cll))),
        y=results_cll,
        error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=std_cll,
            visible=True)
    ))

fig.add_annotation(x=0, y=results_cll[0],showarrow=False,
            text="-0.695026",yshift=-10, xshift=10,
            font=dict(color='rgba(152, 0, 0, .8)'))

fig.add_annotation(x=1, y=results_cll[1],showarrow=False,
            text="-0.6321996",yshift=10, xshift=-35,
            font=dict(color='rgba(152, 0, 0, .8)'))

fig.add_annotation(x=len(results_cll), y=results_cll[-1],showarrow=False,
            text="-0.6095830",yshift=10, xshift=-5,
            font=dict(color='rgba(152, 0, 0, .8)'))

fig.update_layout(title='CLL ao longo das gerações')
fig.update_layout(
    xaxis_title="geracoes",
    yaxis_title="CLL",
)

fig.show()

In [83]:
fig = go.Figure()

fig.add_trace(go.Scatter(
        x=list(range(0, len(results_auc_pr))),
        y=results_rec,name='Recall',
        error_y=dict(
            type='data', # value of error bar given in data coordinates
            array=std_rec,
            visible=True)
    ))


fig.add_annotation(x=0, y=results_rec[0],showarrow=False,
            text="0.0",yshift=10, xshift=-30,
            font=dict(color='rgba(152, 0, 0, .8)'))

fig.add_annotation(x=len(results_rec), y=results_rec[-1],showarrow=False,
            text="0.620407",yshift=-10, xshift=5,
            font=dict(color='rgba(152, 0, 0, .8)'))


fig.add_annotation(x=1, y=results_rec[1],showarrow=False,
            text="0.620407",yshift=-10, xshift=-40,
            font=dict(color='rgba(152, 0, 0, .8)'))



fig.update_layout(title='Recall ao longo das gerações')
fig.update_layout(
    xaxis_title="geracoes",
)

fig.show()
